## Split training dataset to train and valid dataset (70/20/10)

In [5]:
import json

# Path to your annotation file
ann_file = r"data\merged_dataset\train\_annotations.coco.json"

with open(ann_file, "r") as f:
    coco = json.load(f)

print("📂 Categories in dataset:")
for cat in coco["categories"]:
    print(f"ID: {cat['id']}, Name: {cat['name']}")

print("\n👉 Total categories:", len(coco["categories"]))


📂 Categories in dataset:
ID: 0, Name: three_wheels
ID: 1, Name: bus
ID: 2, Name: car
ID: 3, Name: tractor
ID: 4, Name: truck
ID: 5, Name: ambulance
ID: 6, Name: van
ID: 7, Name: motorbikes

👉 Total categories: 8


## Training on Merge Dataset

In [2]:
from rfdetr import RFDETRMedium

# Initialize model (medium backbone version of RF-DETR)
model = RFDETRMedium()

# Train on your dataset
model.train(
    dataset_dir=r"data\merged_dataset",  # path to your dataset
    epochs=50,            # number of training epochs
    batch_size=4,         # number of samples per batch
    grad_accum_steps=4,   # accumulate gradients to simulate a larger batch
    lr=1e-4,              # learning rate
    output_dir=r"./output_rfdetr_merge_dataset"  # where checkpoints & logs will be saved
)


Using a different number of positional encodings than DINOv2, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
Using patch size 16 instead of 14, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
Loading pretrain weights


num_classes mismatch: model has 90 classes, but your dataset has 8 classes
reinitializing your detection head with 8 classes.


Unable to initialize TensorBoard. Logging is turned off for this session.  Run 'pip install tensorboard' to enable logging.
Not using distributed mode
git:
  sha: N/A, status: clean, branch: N/A

Namespace(num_classes=8, grad_accum_steps=4, amp=True, lr=0.0001, lr_encoder=0.00015, batch_size=4, weight_decay=0.0001, epochs=50, lr_drop=100, clip_max_norm=0.1, lr_vit_layer_decay=0.8, lr_component_decay=0.7, do_benchmark=False, dropout=0, drop_path=0.0, drop_mode='standard', drop_schedule='constant', cutoff_epoch=0, pretrained_encoder=None, pretrain_weights='rf-detr-medium.pth', pretrain_exclude_keys=None, pretrain_keys_modify_to_load=None, pretrained_distiller=None, encoder='dinov2_windowed_small', vit_encoder_num_layers=12, window_block_indexes=None, position_embedding='sine', out_feature_indexes=[3, 6, 9, 12], freeze_encoder=False, layer_norm=True, rms_norm=False, backbone_lora=False, force_no_pretrain=False, dec_layers=4, dim_feedforward=2048, hidden_dim=256, sa_nheads=8, ca_nheads=16,